<a href="https://colab.research.google.com/github/GYelejin/knowledgedb/blob/main/titanic_writeup_by_gy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Работа с библиотекой Pandas.

На этой практике я поучаствую в соревновании по машинному обучению. Ссылка на само соревнование: https://www.kaggle.com/c/titanic

# Скачаем данные
Для начала, работая с данными с платформы Kaggle, нам необходимо их скачать. Вы можете загрузить данные  прямо с платформы

In [2]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
! mkdir ~/.kaggle

In [4]:
! mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle  competitions download -c titanic && unzip "/content/titanic.zip"

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 20.3MB/s]
Archive:  /content/titanic.zip
  inflating: gender_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


# Предобработка тренировочных данных (и тестовых)

Здесь у меня задача избавиться от не числовых данных по типу строк. А так же необходимо сгрупировать и создать 

* Рассмотрим данные 
* Заполним пропуски
* Найдем строчные и переведем их в числа
* Если нельзя перевезти то попробуем создать группы на основе строк
* Удалим не нужные данные 

In [112]:
import numpy as np
import pandas as pd
train_dataset = pd.read_csv('train.csv')
test_dataset = pd.read_csv('test.csv')
datasets = [train_dataset, test_dataset]

In [113]:
for dataset in datasets:
    dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass  

In [114]:
train_dataset.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [115]:
train_dataset.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

### 1.1 Name
Использую приставку из поля **"Name"** , которая указывает на социальный статус (Женатость к примеру) назову новою фитчу **"SocialStatus"**

In [116]:
for dataset in datasets:
    dataset['SocialStatus'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

In [117]:
title_mapping = {"Mr": 0, "Miss": 1, "Mrs": 2, 
                 "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,"Countess": 3,
                 "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3, "Dona" : 3, "Mme": 3,"Capt": 3,"Sir": 3 }
for dataset in datasets:
    dataset['SocialStatus'] = dataset['SocialStatus'].map(title_mapping)

### 1.2 Age

Использую поле "SocialStatus" , которая указывает на социальный статус (Женатость к примеру) и заполню пробелы фитчи "Age" на основании медианы возроста групп по социальному статусу 

In [118]:
for dataset  in datasets:
  dataset ["Age"].fillna(dataset .groupby('SocialStatus')["Age"].transform("median"), inplace=True)

In [119]:
for dataset in datasets:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 26), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 26) & (dataset['Age'] <= 36), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 36) & (dataset['Age'] <= 62), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 62, 'Age'] = 4

### 1.3 Sex
Перобразую строчку которая указывает на пол в число

In [120]:
sex_mapping = {"male": 0, "female": 1}
for dataset in datasets:
    dataset['Sex'] = dataset['Sex'].map(sex_mapping)

### 1.3 Embarked
Порт высадки C = Шербур, Q = Квинстаун, S = Саутгемптон
Заменить буквы на числа

In [121]:
embarked_mapping = {"S": 0, "C": 1, "Q": 2}
for dataset in datasets:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_mapping)

In [122]:
for dataset in datasets:
  dataset['Embarked'].fillna(dataset.groupby("Pclass")['Embarked'].transform("median"), inplace=True)

### 1.4 Fare

Стоимость проезда пассажиров, очевидно, зависит класса билетов, поэтому заполню пропуски на основании медианы каждого класса

In [123]:
for dataset in datasets:
  dataset["Fare"].fillna(dataset.groupby("Pclass")["Fare"].transform("median"), inplace=True)

Разделим на группы

In [124]:
for dataset in datasets:
  dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0
  dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 1
  dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2
  dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3

### 1.5 Cabin

Номер кабины состоит из буквы и чисел, числа нам ничего не скажут, а вот буква зависит от расположения и класса
это сильно влияет на живучесть))) 

In [125]:
for dataset in datasets:
    dataset['Cabin'] = dataset['Cabin'].str[:1]

Разделим на группы

In [126]:
cabin_mapping = {"A": 0, "B": 0.4, "C": 0.8, "D": 1.2, "E": 1.6, "F": 2, "G": 2.4, "T": 2.8}
for dataset in datasets:
    dataset['Cabin'] = dataset['Cabin'].map(cabin_mapping)

Заполню пропуски "Cabin" медианой для каждого "Pclass"

In [127]:
for dataset in datasets:
  dataset["Cabin"].fillna(dataset.groupby("Pclass")["Cabin"].transform("median"), inplace=True)


### 1.6 SibSP и Parch
Фитчи отвечают за родственников на борту
из них можно создать новую фитчу про размер семьи

In [128]:
for dataset in datasets:
  dataset["FamilySize"] = dataset["SibSp"] + dataset["Parch"] + 1

In [129]:
family_mapping = {1: 0, 2: 0.4, 3: 0.8, 4: 1.2, 5: 1.6, 6: 2, 7: 2.4, 8: 2.8, 9: 3.2, 10: 3.6, 11: 4}
for dataset in datasets:
    dataset['FamilySize'] = dataset['FamilySize'].map(family_mapping)

### 1.?
Удалим бесполезные фитчи

In [130]:
features_drop = ['Ticket', 'SibSp', 'Parch', 'Name', 'PassengerId']
passenger_id = test_dataset['PassengerId']
train_dataset = train_dataset.drop(features_drop, axis=1)
data = train_dataset.drop(["Survived"], axis=1)
target = train_dataset["Survived"]
test_dataset = test_dataset.drop(features_drop, axis=1)


In [131]:
for datainfo in (train_dataset, data, test_dataset):
  display(datainfo)

,Survived,Pclass,Sex,Age,Fare,Cabin,Embarked,SocialStatus,FamilySize
0,0,3,0,1.0,0.0,2.0,0.0,0,0.4
1,1,1,1,3.0,2.0,0.8,1.0,2,0.4
2,1,3,1,1.0,0.0,2.0,0.0,1,0.0
3,1,1,1,2.0,2.0,0.8,0.0,2,0.4
4,0,3,0,2.0,0.0,2.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...
886,0,2,0,2.0,0.0,1.8,0.0,3,0.0
887,1,1,1,1.0,1.0,0.4,0.0,1,0.0
888,0,3,1,1.0,1.0,2.0,0.0,1,1.2
889,1,1,0,1.0,1.0,0.8,1.0,0,0.0


,Pclass,Sex,Age,Fare,Cabin,Embarked,SocialStatus,FamilySize
0,3,0,1.0,0.0,2.0,0.0,0,0.4
1,1,1,3.0,2.0,0.8,1.0,2,0.4
2,3,1,1.0,0.0,2.0,0.0,1,0.0
3,1,1,2.0,2.0,0.8,0.0,2,0.4
4,3,0,2.0,0.0,2.0,0.0,0,0.0
...,...,...,...,...,...,...,...,...
886,2,0,2.0,0.0,1.8,0.0,3,0.0
887,1,1,1.0,1.0,0.4,0.0,1,0.0
888,3,1,1.0,1.0,2.0,0.0,1,1.2
889,1,0,1.0,1.0,0.8,1.0,0,0.0


,Pclass,Sex,Age,Fare,Cabin,Embarked,SocialStatus,FamilySize
0,3,0,2.0,0.0,2.0,2,0,0.0
1,3,1,3.0,0.0,2.0,0,2,0.4
2,2,0,3.0,0.0,2.0,2,0,0.0
3,3,0,2.0,0.0,2.0,0,0,0.0
4,3,1,1.0,0.0,2.0,0,2,0.8
...,...,...,...,...,...,...,...,...
413,3,0,2.0,0.0,2.0,0,0,0.0
414,1,1,3.0,3.0,0.8,1,3,0.0
415,3,0,3.0,0.0,2.0,0,0,0.0
416,3,0,2.0,0.0,2.0,0,0,0.0


#Обучение

In [132]:
import sklearn.neighbors 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=5)

cross_val_scores = cross_val_score(knn, data, target, cv=5, scoring='accuracy')
print(cross_val_scores)
print(np.mean(cross_val_scores))

[0.79329609 0.78651685 0.84831461 0.79775281 0.83146067]
0.8114682066411399


In [133]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

cross_val_scores = cross_val_score(lr, data, target, cv=5, scoring='accuracy')
print(cross_val_scores)
print(np.mean(cross_val_scores))

[0.82122905 0.82022472 0.80898876 0.81460674 0.83146067]
0.8193019898311468


In [134]:
lr.fit(data,target)
knn.fit(data,target)

KNeighborsClassifier()

# Предсказания

In [135]:
y_pred_lr = lr.predict(test_dataset)
y_pred_knn = knn.predict(test_dataset)

# Результат

In [136]:
submission = pd.DataFrame(y_pred_knn, columns=['Survived'])
submission['PassengerId'] = passenger_id
submission = submission[['PassengerId', 'Survived']]
submission.to_csv('submission_titanic.csv', index=None)

In [137]:
submission

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [138]:
!kaggle competitions submit -c titanic -f submission_titanic.csv -m "New one"

100% 2.77k/2.77k [00:03<00:00, 719B/s]
Successfully submitted to Titanic - Machine Learning from Disaster